# Skip this. Save it for a challenge later. Trying to find all lat and long missing and approximating it

In [100]:
# na_gps_rows = df[(df['GPS_LONGITUDE'].isna()) | (df['GPS_LATITUDE'].isna())]
# vid_list_nans = na_gps_rows['VEHICLE_ID'].drop_duplicates(keep='first').tolist()

In [101]:
# def fill_lat_long(row):
#     if pd.isna(row['GPS_LATITUDE']) or pd.isna(row['GPS_LONGITUDE']):
#         prev_lat = df.at[row.name - 1, 'GPS_LATITUDE']
#         prev_long = df.at[row.name - 1, 'GPS_LONGITUDE']
#         a_distance = df.at[row.name - 1, 'METERS']
#         b_distance = row['METERS']
        
#         filled_lat, filled_long = lat_long_filler((prev_lat, prev_long), a_distance, b_distance)
#         return pd.Series({'GPS_LATITUDE': filled_lat, 'GPS_LONGITUDE': filled_long})
#     else:
#         return row[['GPS_LATITUDE', 'GPS_LONGITUDE']]

In [102]:
# for id in vid_list_nans:
#     cur_vid_rows = df[df['VEHICLE_ID'] == id]
#     if (cur_vid_rows['GPS_LATITUDE'].isna().any()) or (cur_vid_rows['GPS_LONGITUDE'].isna().any()):
#         caught_series = cur_vid_rows.apply(fill_lat_long, axis=1)
#         df[df['VEHICLE_ID'] == id] = caught_series

# Setup

In [103]:
from src.subpipe.validate import ValidateBusData
import logging, os, sys, json
import pandas as pd

from src.utils.utils import (
    DATA_MONTH_DAY,
    SUBSCRIBER_DATA_PATH_JSON,
    SUBSCRIBER_FOLDER,
    curr_time_micro,
    sub_logger,
    lat_long_filler
)

logging.basicConfig(
        format="",
        filename=f"logs/notebook-{DATA_MONTH_DAY}.log",
        encoding="utf-8",
        filemode="a",
        level=logging.INFO,
)

# Code For Transform

In [104]:
df = pd.read_json(os.path.join(SUBSCRIBER_FOLDER, "04-12.json"))
df.size

2953540

In [105]:
df = df.sort_values(["VEHICLE_ID", "ACT_TIME"], ascending=True)
df.size

2953540

In [106]:
df = df.dropna()
df.size

2952180

In [107]:
df = df.reset_index(level=0)
df.size

3247398

In [108]:
df = df.drop(columns=['index', 'GPS_SATELLITES', 'GPS_HDOP'])
df.size

2361744

In [109]:
df.insert(5, 'TIMESTAMP', 0)

In [110]:
date_format = "%d%b%Y:%H:%M:%S"

In [111]:
opd_sec: pd.Timestamp = pd.to_datetime(pd.to_datetime(df['OPD_DATE'], format=date_format), unit='s')

In [112]:
td: pd.Timedelta = pd.to_timedelta(df['ACT_TIME'], unit='sec')

In [113]:
df['TIMESTAMP'] = opd_sec + td
test_more = df.loc[241350:241450]

In [114]:
test_more

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,TIMESTAMP,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
241350,218982530,218982540,13DEC2022:00:00:00,3754,90474,2022-12-13 22:23:41,80621,-122.668513,45.503967
241351,218982530,218982540,13DEC2022:00:00:00,3754,90532,2022-12-13 22:23:46,80626,-122.667967,45.504323
241352,218982530,218982540,13DEC2022:00:00:00,3754,90589,2022-12-13 22:23:51,80631,-122.667418,45.504682
241353,218982530,218982540,13DEC2022:00:00:00,3754,90648,2022-12-13 22:23:56,80636,-122.666855,45.505048
241354,218982530,218982540,13DEC2022:00:00:00,3754,90709,2022-12-13 22:24:01,80641,-122.666275,45.505427
...,...,...,...,...,...,...,...,...,...
241446,218982530,218982549,13DEC2022:00:00:00,3754,94663,2022-12-13 22:35:05,81305,-122.617293,45.505280
241447,218982530,218982550,13DEC2022:00:00:00,3754,94707,2022-12-13 22:35:10,81310,-122.616725,45.505282
241448,218982530,218982550,13DEC2022:00:00:00,3754,94755,2022-12-13 22:35:15,81315,-122.616100,45.505283
241449,218982530,218982550,13DEC2022:00:00:00,3754,94802,2022-12-13 22:35:20,81320,-122.615483,45.505288


In [115]:
df.drop(columns=['OPD_DATE', 'ACT_TIME'], inplace=True)

df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,2022-12-13 06:18:00,-122.843617,45.504060
1,219000623,219000624,2905,47,2022-12-13 06:18:05,-122.843743,45.504095
2,219000623,219000624,2905,54,2022-12-13 06:18:10,-122.843828,45.504118
3,219000623,219000626,2905,181,2022-12-13 06:19:00,-122.843408,45.505170
4,219000623,219000626,2905,196,2022-12-13 06:19:05,-122.843285,45.505283


In [116]:
df.insert(4, 'SPEED', 0)
df.insert(4, 'dMETERS', df['METERS'].diff())
df.insert(4, 'dTIMESTAMP', df['TIMESTAMP'].diff())
df.head()


,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,dTIMESTAMP,dMETERS,SPEED,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,NaT,NaN,0,2022-12-13 06:18:00,-122.843617,45.504060
1,219000623,219000624,2905,47,0 days 00:00:05,10.0,0,2022-12-13 06:18:05,-122.843743,45.504095
2,219000623,219000624,2905,54,0 days 00:00:05,7.0,0,2022-12-13 06:18:10,-122.843828,45.504118
3,219000623,219000626,2905,181,0 days 00:00:50,127.0,0,2022-12-13 06:19:00,-122.843408,45.505170
4,219000623,219000626,2905,196,0 days 00:00:05,15.0,0,2022-12-13 06:19:05,-122.843285,45.505283


In [117]:
df['SPEED'] = df.apply((lambda df: df['dMETERS']/df['dTIMESTAMP'].total_seconds()), axis=1)
df.drop(columns=['dTIMESTAMP', 'dMETERS'], inplace=True)


In [118]:
df.head()
other_new = df.loc[241350:241450]

In [119]:
vid_list = df['VEHICLE_ID'].drop_duplicates(keep='first').tolist()

modified_index = 0
for id in vid_list:
    curr_data = df[df['VEHICLE_ID'] == id]
    curr_data.loc[modified_index, 'SPEED'] = curr_data.loc[modified_index+1, 'SPEED']
    df[df['VEHICLE_ID'] == id] = curr_data
    modified_index += curr_data.shape[0]
    
    

In [120]:
df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,VEHICLE_ID,METERS,SPEED,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,219000623,219000624,2905,37,2.00,2022-12-13 06:18:00,-122.843617,45.504060
1,219000623,219000624,2905,47,2.00,2022-12-13 06:18:05,-122.843743,45.504095
2,219000623,219000624,2905,54,1.40,2022-12-13 06:18:10,-122.843828,45.504118
3,219000623,219000626,2905,181,2.54,2022-12-13 06:19:00,-122.843408,45.505170
4,219000623,219000626,2905,196,3.00,2022-12-13 06:19:05,-122.843285,45.505283


In [121]:
kmh_data = df["SPEED"] * 3.6
df['SPEED'] = kmh_data
df = df[df["SPEED"] < (160 / 3.6)]